##### Copyright 2019 The TensorFlow Authors. (Traducción al español por Juan José Rojas Constaín X & Github: @jrconstain)

In [2]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Generación de texto con RNN (Redes Neuronales Recurrentes)

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/text/tutorials/text_generation"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/text/blob/master/docs/tutorials/text_generation.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/text/blob/master/docs/tutorials/text_generation.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/text/docs/tutorials/text_generation.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

Este tutorial demuestra cómo generar texto usando una Red Neuronal Recurrente (RNN) basada en caracteres (letras, espacios, etc). Se trabajará con un conjunto de datos de los escritos de Shakespeare's tomados del artículo de Andrej Karpathy's ["The Unreasonable Effectiveness of Recurrent Neural Networks"](http://karpathy.github.io/2015/05/21/rnn-effectiveness/) (La irrazonable efectividad de las Redes Neuronales Recurrentes). Dada una secuencia de caracteres de estos datos (p.e. "Shakespear"), se entrenará un modelo para predecir el próximo caracter en la secuencia ("e"). Cadenas de texto más largas pueden ser generadas ejecutando el modelo repetidamente.

Nota: Habilite la aceleración de GPU para ejecutar más rápido este cuadero. En Colab: *Entorno de ejecución > Cambiar tipo de entorno de ejecución > Acelerador por hardware > GPU*.

Este tutorial incluye código ejecutable implementado usando ['tf.keras'](https://www.tensorflow.org/guide/keras/sequential_model) y ['eager execution'](https://www.tensorflow.org/guide/eager). La siguiente es una muestra del resultado, cuando el modelo en este tutorial se entrenó durante 30 épocas y se inició con el indicador "Q":

<pre>
QUEENE:
I had thought thou hadst a Roman; for the oracle,
Thus by All bids the man against the word,
Which are so weak of care, by old care done;
Your children were in your holy love,
And the precipitation through the bleeding throne.

BISHOP OF ELY:
Marry, and will, my lord, to weep in such a one were prettiest;
Yet now I was adopted heir
Of the world's lamentable day,
To watch the next way with his father with his face?

ESCALUS:
The cause why then we are all resolved more sons.

VOLUMNIA:
O, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, it is no sin it should be dead,
And love and pale as any will to that word.

QUEEN ELIZABETH:
But how long have I heard the soul for this world,
And show his hands of life be proved to stand.

PETRUCHIO:
I say he look'd on, if I must be content
To stay him from the fatal of our country's bliss.
His lordship pluck'd from this sentence then for prey,
And then let us twain, being the moon,
were she such a case as fills m
</pre>

While some of the sentences are grammatical, most do not make sense. The model has not learned the meaning of words, but consider:

* The model is character-based. When training started, the model did not know how to spell an English word, or that words were even a unit of text.

* The structure of the output resembles a play—blocks of text generally begin with a speaker name, in all capital letters similar to the dataset.

* As demonstrated below, the model is trained on small batches of text (100 characters each), and is still able to generate a longer sequence of text with coherent structure.

## Configuración

##Importar TensorFlow, otras librerías y datos

In [2]:
# Se importa la biblioteca TensorFlow, que es una plataforma de código abierto para machine learning.
import tensorflow as tf

# Se importa la biblioteca numpy, que es una biblioteca para el manejo de arrays y matrices, así como
# la realización de operaciones matemáticas en ellos. Es ampliamente utilizada en ciencia de datos y computación científica.
import numpy as np

# Se importa el módulo os, que provee una manera de usar funcionalidades dependientes del sistema operativo,
# como leer o escribir en el sistema de archivos.
import os

# Se importa el módulo time, que provee funciones para trabajar con tiempos, como pausar la ejecución y medir el tiempo.
import time


### Descarga el conjunto de datos 'Shakespeare'

Cambiar la siguiente línea para correr este código con datos propios.

In [3]:
# Se define la variable 'path_to_file'.
# La función 'get_file' de 'tf.keras.utils' se utiliza para descargar un archivo desde una URL y almacenarlo localmente.
# Si el archivo ya ha sido descargado previamente, la función lo recuperará desde la caché en lugar de volver a descargarlo.
# El primer argumento ('shakespeare.txt') es el nombre que se le dará al archivo descargado.
# El segundo argumento es la URL desde donde se descargará el archivo.
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')


1115394/1115394 [==============================] - 0s 0us/step


### Lectura de datos

Una mirada dentro del texto:

In [4]:
# Abre el archivo en ruta 'path_to_file' en modo binario ('rb') para lectura.
# Lee el contenido completo del archivo con 'read()'.
# Decodifica el contenido binario a cadenas de texto usando UTF-8 como codificación
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')

# Imprime longitud del texto, que es el número total de caracteres en él.
# Utiliza una f-string (cadena formateada) para insertar el valor de 'len(text)' dentro del mensaje.
print(f'Longitud del texto: {len(text)} caracteres')

Longitud del texto: 1115394 caracteres


In [5]:
# Imprimir los primeros 300 caracters en el texto
print(text[:300])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us


In [6]:
# Crea una lista de caracteres únicos en el texto.

# Función 'set()' obteniee el conjunto de caracteres únicos del texto (eliminando duplicados).
# Función 'sorted()' ordena esos caracteres en una lista.
# Crea variable 'vocab' con todos los caracteres diferentes que aparecen en el texto
vocab = sorted(set(text))

# Imprime el número total de caracteres únicos en el texto, lo que da una idea de la diversidad de caracteres en el texto.
print(f'{len(vocab)} caracteres únicos')


65 caracteres únicos


## Procesamiento de texto

### Vectorizar el texto

Antes del entrenamiento, se deben convertir las secuencias de texto (strings) en una representación numérica.

La capa `tf.keras.layers.StringLookup` convierte cada caracter en un ID numerico. Solo que primero se debe dividir el texto en tokens.

"Capa" se refiere a un conjunto de neuronas que procesan datos en una red neuronal. En RN y DL, una capa es una estructura fundamental que toma una entrada, realiza alguna transformación o cálculo sobre ella y produce una salida.

`tf.keras.layers.StringLookup` es una capa especializada en Keras que convierte cadenas en índices enteros basados en un vocabulario predefinido. No es una "capa" en el sentido tradicional de procesar características, sino más bien una herramienta para la preparación de datos.

In [7]:
# Se define una lista 'example_texts' con dos cadenas de caracteres: 'abcdefg' y 'xyz'.
example_texts = ['abcdefg', 'xyz']

# Utilizando la función 'unicode_split' de 'tf.strings', se descompone cada cadena en la lista 'example_texts' en sus caracteres individuales.
# El argumento 'input_encoding' especifica la codificación de los caracteres en las cadenas; en este caso, se usa 'UTF-8'.
chars = tf.strings.unicode_split(example_texts, input_encoding='UTF-8')

# Se muestra el resultado de la operación anterior.
chars


<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

Ahora se crea la capa `tf.keras.layers.StringLookup`:

In [8]:
# Se crea una instancia de la capa 'StringLookup' de Keras, la cual convierte cadenas en índices enteros.
# Esta capa es útil cuando se trabaja con texto y se desea convertir palabras o caracteres en números que puedan ser
# procesados por un modelo de machine learning.
#
# Parámetros:
#   - vocabulary: Especifica el vocabulario que se utilizará para la conversión. En este caso, se pasa la lista 'vocab'
#     que contiene los caracteres únicos del texto.
#   - mask_token: Al establecerlo en 'None', indica que no se desea usar un token de máscara. Un token de máscara
#     generalmente se usa para representar valores desconocidos o faltantes, pero aquí se omite.

ids_from_chars = tf.keras.layers.StringLookup(
    vocabulary=list(vocab), mask_token=None)


Se convierte de tokens a índices únicos de cada caracter:

In [9]:
# Usando la capa 'ids_from_chars' que se creó previamente, se convierten los caracteres en 'chars' a sus respectivos índices enteros.
# La capa 'ids_from_chars' fue definida para mapear cada carácter en el vocabulario a un índice entero único.
# Al llamar a esta capa con la entrada 'chars' (que contiene las cadenas divididas en caracteres),
# se obtiene un tensor de índices enteros que representan esos caracteres.
ids = ids_from_chars(chars)

# Se muestra el tensor resultante 'ids'. Este tensor contiene los índices enteros que corresponden a los caracteres en 'chars'.
ids


<tf.RaggedTensor [[40, 41, 42, 43, 44, 45, 46], [63, 64, 65]]>

Dado que el objetivo de este tutorial es generar texto, después de que el modelo produce una salida en forma de índices, será importante invertir esta representación a cadenas legibles por humanos.

Para ello, puedes usar tf.keras.layers.StringLookup(..., invert=True).
Al establecer el parámetro invert en True, le dices a la capa que quieres convertir índices a cadenas, en lugar de cadenas a índices.

**Nota:**
Aquí, en lugar de pasar el vocabulario original generado con `sorted(set(text))`, utiliza el método `get_vocabulary()` de la capa `tf.keras.layers.StringLookup` para que el token `[UNK]` se establezca de la misma manera.

Cuando inviertes la representación, es crucial que uses el mismo vocabulario que se utilizó para la conversión original. La capa StringLookup puede añadir automáticamente un token especial llamado `[UNK]` para manejar caracteres o palabras desconocidas. El método `get_vocabulary()` proporcionará el vocabulario exacto que la capa está utilizando, incluyendo cualquier token especial como `[UNK]`.

In [10]:
# Crea una instancia de la capa 'StringLookup' de Keras, configurada para realizar la operación inversa: convertir índices enteros en cadenas.

# Parámetros:
#   - vocabulary: Especifica el vocabulario que utilizará la conversión, el método 'get_vocabulary()' de la capa 'ids_from_chars' garantiza que se tenga en cuenta cualquier token adicional como '[UNK]'
#   - invert: 'True' indica que deseamos invertir la operación, i.e. convertir índices enteros en cadenas, y no al revés.
#   - mask_token:'None' indica que no se desea usar un token de máscara para representar valores desconocidos o faltantes.
chars_from_ids = tf.keras.layers.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)


Esta capa recupera caracteres del vector de índices, y los retorna como un `tf.RaggedTensor` de caracteres:

In [11]:
# variable cars se crea usando la capa recien creada para procesar tensor 'ids'
chars = chars_from_ids(ids)
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

Se puede usar `tf.strings.reduce_join` para juntar los carácteres en secuencias.

In [12]:
# Función 'reduce_join' de 'tf.strings' concatena 'chars' a lo largo de la última dimensión (axis=-1)
# Se convierte el resultado en un array de numpy con '.numpy()'.
tf.strings.reduce_join(chars, axis=-1).numpy()

array([b'abcdefg', b'xyz'], dtype=object)

In [13]:
# Define función 'text_from_ids' con objetivo de convertir un tensor de 'ids'  en texto, basado en el mapeo inverso de la capa 'chars_from_ids'.
def text_from_ids(ids):
    # Utiliza directamente la capa 'chars_from_ids' para convertir el tensor de índices 'ids' en tensor de caracteres.
    # Con 'tf.strings.reduce_join' concatena caracteres a lo largo de la última dimensión para formar tensor de cadenas de texto.
    return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)



### La tarea de predicción

Dado un caracter (letra) o una secuencia de caracteres, ¿cuál es el próximo caracter con mayor probabilidad? Esta es la tarea para la cual se está entrenando el modelo. La entrada al modelo será una secuencia de caracteres, y se lo entrenará para predecir una salida: el siguiente carácter en cada iteración.

Dado que las RNNs mantienen un estado interno que depende de los elementos vistos previamente, considerando todos los caracteres procesados hasta el momento, ¿cuál es el siguiente caracter?


### Crea ejemplos y objetivos para el entrenamiento

Se divide el texto en secuencias ejemplo. Cada secuencia de entrada contendrá un número de caracteres dado por `seq_length`.

Para cada "input", los "targets" contienen la misma longitud de texto, excepto que se desplaza un carácter a la derecha.

Divide el texto en fragmentos con `seq_length+1` caracteres. Por ejemplo, supongamos que `seq_length` es 4 y nuestro texto es "Hola". La secuencia input sería "Hola", y la secuencia target sería "ola ".

Para hacer esto, primero utiliza la función `tf.data.Dataset.from_tensor_slices` para convertir el vector de texto en una secuencia de índices de caracteres.

In [14]:
# 'unicode_split' de 'tf.strings' descompone 'text' en caracteres individuales usando Unicode 'UTF-8'.
# 'ids_from_chars' convierte caracteres en índices enteros. Se almacena tensor resultante de índices enteros en 'all_ids'.
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))

# Tensor 'all_ids' contiene los índices enteros que representan cada carácter en el texto original.
all_ids


<tf.Tensor: shape=(1115394,), dtype=int64, numpy=array([19, 48, 57, ..., 46,  9,  1])>

In [15]:
# Crea objeto 'Dataset' de TensorFlow a partir del tensor 'all_ids'.
# Función 'from_tensor_slices' toma tensor 'all_ids' y lo convierte en un 'Dataset', donde cada elemento es un índice entero del tensor original.
# 'Dataset' facilita la manipulación, transformación y batching de los datos para el entrenamiento de modelos.
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)


In [16]:
# Itera sobre los primeros 13 elementos del objeto 'Dataset' 'ids_dataset'.
# Cada índice lo convierte de vuelta a su carácter correspondiente usando la capa 'chars_from_ids'.
# Decodifica el carácter resultante desde su representación binaria a una cadena UTF-8 y lo imprime.
for ids in ids_dataset.take(13):
    print(chars_from_ids(ids).numpy().decode('utf-8'))

F
i
r
s
t
 
C
i
t
i
z
e
n


In [17]:
# establece variable 'seq_length' en 100. Cada secuencia ejemplo estará compuesta de 100 carácteres.
seq_length = 100


El método `batch` permite convertir fácilmente estos carácteres individuales en secuencias del tamaño deseado.

In [18]:
# Variable 'sequences' contendrá subconjuntos ("batches") de 'ids_dataset', de 101 elemento de largo, utilizando la función '.batch()'.
# 'seq_length+1' establece el tamaño de los parches (100+1) y 'drop_remainder=True' asegura descarta cualquier remanente que no permita cumplir con ese tamaño (todos los subconjuntos quedan de 101 elementos).
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

#Itera sobre la primera secuencia de 101 elementos y la imprime como caracteres
for seq in sequences.take(1):
  print(chars_from_ids(seq))

tf.Tensor(
[b'F' b'i' b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z' b'e' b'n' b':'
 b'\n' b'B' b'e' b'f' b'o' b'r' b'e' b' ' b'w' b'e' b' ' b'p' b'r' b'o'
 b'c' b'e' b'e' b'd' b' ' b'a' b'n' b'y' b' ' b'f' b'u' b'r' b't' b'h'
 b'e' b'r' b',' b' ' b'h' b'e' b'a' b'r' b' ' b'm' b'e' b' ' b's' b'p'
 b'e' b'a' b'k' b'.' b'\n' b'\n' b'A' b'l' b'l' b':' b'\n' b'S' b'p' b'e'
 b'a' b'k' b',' b' ' b's' b'p' b'e' b'a' b'k' b'.' b'\n' b'\n' b'F' b'i'
 b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z' b'e' b'n' b':' b'\n' b'Y'
 b'o' b'u' b' '], shape=(101,), dtype=string)



Es más fácil ver lo que se está haciendo si se unen los tokens de nuevo en secuencias:

In [19]:
# Itera sobre las primeras 5 secuencias y las imprime como cadenas de texto utilizando la función 'text_from_ids' definida en [34]
for seq in sequences.take(5):
  print(text_from_ids(seq).numpy())

b'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
b'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'
b"now Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us ki"
b"ll him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be d"
b'one: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citi'


Para entrenar el modelo se necesitará un conjunto de datos formado por pares `(input, label)`. donde `input` y `label` son secuencias. En cada iteración, el input es el caracter actual y label es el siguiente caracter.

La siguiente función toma una secuencia como argumento, la duplica, y la desplaza para alinear el input y el label para cada iteración:

In [20]:
# se define la función 'split_input_target' que toma una secuencia ('sequence') como argumento y retorna 'input_text' y 'target_text'
# 'input_text' es la sequencia menos el último caracter
# 'target_text' es la sequencia a partir del segundo caracter
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [21]:
# Se aplica funcion a "TensorFlow" como ejemplo:
split_input_target(list("Tensorflow"))

(['T', 'e', 'n', 's', 'o', 'r', 'f', 'l', 'o'],
 ['e', 'n', 's', 'o', 'r', 'f', 'l', 'o', 'w'])

In [22]:
# Utiliza método 'map' para aplicar la función 'split_input_target' a cada elemento del objeto 'Dataset' 'sequences'
# 'dataset' es un nuevo objeto 'Dataset' donde cada elemento es una tupla compuesta por una secuencia de entrada (input) y una secuencia objetivo (target).
dataset = sequences.map(split_input_target)

In [23]:
# Itera la primera tupla en 'dataset' para mostrar las secuencias 'input_example' y 'target_example'
for input_example, target_example in dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())

Input : b'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'
Target: b'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '


### Create training batches

Se uso `tf.data` para dividir el texto en secuencias manejables. Pero antes de alimentar estos datos al modelo, es necesario reordenar aleatoriamente los datos -crucial para asegurar que el modelo no aprenda patrones indeseados basado en el orden de los datos-, y agruparlos en lotes -subconjuntos de datos que se alimenta al modelo en una sola iteración para que el entrenamiento sea más eficiente en GPU's-

In [24]:
# Constante 'BATCH_SIZE' indica el número de secuencias que se incluirán en cada lote durante el entrenamiento (64).
BATCH_SIZE = 64

# 'BUFFER_SIZE' determina el número de elementos que se deben cargar en memoria para ser mezclados.
# Librería tf.data está diseñada para trabajar con secuencias que pueden ser infinitamente largas, por lo que no intenta mezclar toda la secuencia en memoria.
# En su lugar, mantiene un buffer (en este caso de tamaño 10,000) y mezcla elementos dentro de ese buffer.
BUFFER_SIZE = 10000

# Se realizan varias transformaciones en el 'Dataset' 'dataset':
dataset = (
    dataset
    # 'shuffle' mezcla aleatoriamente los elementos del 'Dataset' usando un buffer de tamaño 'BUFFER_SIZE'.
    .shuffle(BUFFER_SIZE)
    # 'batch' agrupa los datos en lotes de tamaño 'BATCH_SIZE'.
    # 'drop_remainder=True' asegura que si un lote no alcanza tamaño 64 (p. e., al final del 'Dataset') se descarte.
    .batch(BATCH_SIZE, drop_remainder=True)
    # 'prefetch' permite que el modelo de entrenamiento y el proceso de carga de datos se solapen, mejorando la eficiencia del entrenamiento.
    # 'tf.data.experimental.AUTOTUNE' permite a TensorFlow decidir automáticamente cuántos lotes deben ser pre-cargados.
    .prefetch(tf.data.experimental.AUTOTUNE))

# Se muestra el objeto 'Dataset' transformado 'dataset'.
dataset

<_PrefetchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

## Construcción del modelo

En esta sección se define el modelo como una subclase de `keras.Model` (Para más detalles vea [Making new Layers and Models via subclassing](https://www.tensorflow.org/guide/keras/custom_layers_and_models)).

Este modelo tiene tres capas:

* `tf.keras.layers.Embedding`:  La capa de entrada. Una tabla de búsqueda entrenable que mapeará cada ID de carácter a un vector con dimensiones  `embedding_dim`
* `tf.keras.layers.GRU`: Un tipo de RNN de tamaño `units=rnn_units` (También puedes usarse una capa LSTM aquí).
* `tf.keras.layers.Dense`:  La capa de salida, con `vocab_size` salidas. Produce un logit para cada carácter en el vocabulario. Estos son la log-verosimilitud de cada carácter según el modelo.

In [25]:
# Se establece la longitud del vocabulario obtenida de 'ids_from_chars'
vocab_size = len(ids_from_chars.get_vocabulary())

# Se establece la dimensión de los vectores de entrada
embedding_dim = 256

# Se establece número de unidades neuronales en la GRU (Gated Recurrent Unit)
rnn_units = 1024

In [26]:
# Se define una clase 'MyModel' que hereda de 'tf.keras.Model', que significa que este modelo personalizado se beneficiará de características y funcionalidades del modelo base de Keras.
class MyModel(tf.keras.Model):

  # Método constructor de la clase.
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    # Se llama al constructor de la clase base 'tf.keras.Model'.
    super().__init__(self)
    # Se define la capa 'embedding' que transformará los índices de caracteres en vectores densos.
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    # Se define la capa 'gru', que es una red neuronal recurrente (GRU).
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,  # La capa devolverá la secuencia completa y no solo el último output.
                                   return_state=True)      # La capa devolverá el estado final además del output.
    # Se define la capa 'dense', que producirá logits para cada carácter en el vocabulario.
    self.dense = tf.keras.layers.Dense(vocab_size)

  # Método 'call' define la lógica de propagación hacia adelante del modelo.
  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    # Se pasa la entrada a través de la capa 'embedding'.
    x = self.embedding(x, training=training)
    # Se verifica si se proporciona un estado inicial. Si no, se obtiene el estado inicial de la capa 'gru'.
    if states is None:
      states = self.gru.get_initial_state(x)
    # Se pasa la salida de la capa 'embedding' a través de la capa 'gru'.
    x, states = self.gru(x, initial_state=states, training=training)
    # Se pasa la salida de la capa 'gru' a través de la capa 'dense'.
    x = self.dense(x, training=training)

  # Si 'return_state' es True, se devuelve tanto la salida como el estado. Si es False, solo se devuelve la salida.
    if return_state:
      return x, states
    else:
      return x

In [27]:
# Se crea una instancia del modelo 'MyModel', proporcionando los parámetros necesarios para su construcción.
model = MyModel(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)


Para cada caracter, el modelo busca el embedding, ejecuta una iteración del GRU con el embedding como entrada, y aplica la capa densa para generar logits que predicen la log-verosimilitud del siguiente caracter:

![A drawing of the data passing through the model](https://github.com/tensorflow/text/blob/master/docs/tutorials/images/text_generation_training.png?raw=1)

**Note:** Para el entrenamiento puede usarse un modelo `keras.Sequential`. Sin embargo, para generar texto, se necesitará gestionar el estado interno de la RNN. Es más sencillo incluir las opciones de entrada y salida del estado desde el principio, que reorganizar la arquitectura del modelo más tarde. Para más detalles, consulta la [Guía de RNN de Keras](https://www.tensorflow.org/guide/keras/rnn#rnn_state_reuse).

## Prueba del modelo

Ahora se corre el modelo para asegurarse de que se comporta de la manera esperada.

Se revisa primero las dimensiones del output:

In [29]:
# Se itera sobre un único lote (batch) del conjunto de datos 'dataset'.
for input_example_batch, target_example_batch in dataset.take(1):

    # Se utiliza el modelo 'model' para obtener las predicciones del lote de entrada 'input_example_batch'.
    example_batch_predictions = model(input_example_batch)

    # Se imprime la forma (shape) de las predicciones obtenidas y una explicación de cada dimensión.
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")


(64, 100, 66) # (batch_size, sequence_length, vocab_size)


En este ejemplo, la longitud de la secuencia de entrada es`100`, pero el modelo puede ser ejecutado con entradas de cualquier longitud:

In [30]:
# Resumen de especificaciones del modelo
model.summary()

Model: "my_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  16896     
                                                                 
 gru (GRU)                   multiple                  3938304   
                                                                 
 dense (Dense)               multiple                  67650     
                                                                 
Total params: 4,022,850
Trainable params: 4,022,850
Non-trainable params: 0
_________________________________________________________________


Para obtener predicciones reales del modelo, se necesita tomar muestras de la distribución de salida, para obtener índices de caracteres reales. Esta distribución está definida por los logits sobre el vocabulario de caracteres.

Nota: Es importante tomar muestras de esta distribución ya que tomar el argmax de la distribución puede hacer que el modelo quede atrapado fácilmente en un bucle

Se prueba para el primer ejemplo en el lote:

In [31]:
# Se utiliza la función 'tf.random.categorical' para muestrear índices a partir de las predicciones del modelo.
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)

# Se elimina una dimensión innecesaria de 'sampled_indices' y se convierte el tensor resultante a un array de NumPy.
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

Esto nos da, en cada iteración, una predicción del siguiente caracter en el índice:

In [32]:
sampled_indices

array([ 1, 11,  2, 30,  7, 64, 12, 59,  2, 33, 18,  8, 52, 20, 13,  8,  3,
       60, 47, 25, 47, 30, 28, 31, 57, 64, 26, 52, 28, 20, 21, 65, 55, 59,
       52, 30, 45, 40, 13,  5, 25, 14, 54, 49, 24, 27, 42, 40, 15, 31, 19,
       19,  9, 31,  4, 30,  0, 30, 24, 16,  3, 55, 20, 42, 43, 21, 45, 58,
       28, 27, 55, 25, 35, 62,  0, 28,  8, 38, 34, 58, 41,  1, 35, 38,  0,
        1, 33, 28, 10, 63, 11, 58,  8, 55, 64,  7, 49, 38, 57, 11])

Se decodifica para observar el texto predicho por este modelo sin entrenamiento:

In [33]:
print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print()
print("Predicción del próximo caracter:\n", text_from_ids(sampled_indices).numpy())

Input:
 b'll will come to nought,\nWhen such bad dealings must be seen in thought.\n\nGLOUCESTER:\nHow now, my lor'

Predicción del próximo caracter:
 b'\n: Q,y;t TE-mG?-!uhLhQORryMmOGHzptmQfa?&LAojKNcaBRFF.R$Q[UNK]QKC!pGcdHfsONpLVw[UNK]O-YUsb\nVY[UNK]\nTO3x:s-py,jYr:'


## Entrenamiento del modelo

En este punto, el problema puede ser tratado como un problema de clasificación estandar. Dado el estado previo de la RNN, y la entrada en dicha iteración, se predice la clase del siguiente caracter.

### Adjunta un optimizador y una función de perdida


La "pérdida" es una medida escalar que indica cuán bien (o mal) está funcionando un modelo en un conjunto de datos específico. Se calcula comparando las predicciones del modelo con las verdaderas etiquetas o valores objetivo. Durante el entrenamiento de un modelo, el objetivo principal es minimizar esta pérdida, ajustando los parámetros del modelo (como los pesos y sesgos en una red neuronal).

La función de perdida estandar `tf.keras.losses.sparse_categorical_crossentropy` funciona en este caso porque es se aplica a través de la última dimensión de las predicciones.

Dado que tu modelo devuelve logits, necesitas establecer el indicador  `from_logits`.


In [35]:
# Define la función de pérdida 'SparseCategoricalCrossentropy', especificando que las predicciones proporcionadas serán logits.
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [36]:
# Se calcula la pérdida promedio para un lote de ejemplos, utilizando las predicciones del modelo y las verdaderas etiquetas objetivo.
example_batch_mean_loss = loss(target_example_batch, example_batch_predictions)

# Se imprime la forma de las predicciones y la pérdida promedio calculada para el lote.
print("Dimensiones de la predicción: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Perdida promedio:        ", example_batch_mean_loss)

Dimensiones de la predicción:  (64, 100, 66)  # (batch_size, sequence_length, vocab_size)
Perdida promedio:         tf.Tensor(4.1904016, shape=(), dtype=float32)


Un modelo inicializado recientemente no debería estar muy seguro de si mismo, por lo que los logits resultado deberían todos tener magnitudes ssimilares.

Para confirmar esto, se puede revisar que el exponente de la perdida promedio es aproximadamente igual al tamaño del vocabulario. Una perdida mucho mayor significa que el modelo está seguro de sus respuestas erroneas, y está mal inicializado:

In [38]:
# se eleva a la e la perdida promedio del conjunto ejemplo, que da aproximadamente igual al tamaño del vocabulario (66)
tf.exp(example_batch_mean_loss).numpy()

66.04931

Se configura el procedimiento de entrenamiento con el método `tf.keras.Model.compile`. Se usa `tf.keras.optimizers.Adam` con argumentos default y la funnción de perdida.

In [40]:
# Se compila el modelo especificando el optimizador y la función de pérdida.
model.compile(optimizer='adam', loss=loss)

### Configurar puntos de control

Se usa un `tf.keras.callbacks.ModelCheckpoint` para asegurar que se guardan puntos de control durante el entrenamiento:

In [41]:
# Se establece el directorio donde se guardaran los puntos de control
checkpoint_dir = './training_checkpoints'
# Se configura el nombre de los puntos de control  (época -epoch- se refiere a un ciclo completo a través del conjunto de datos de entrenamiento.)
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

# Se configura el callback para guardar los pesos del modelo después de cada época.
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

### Ejecución del entrenamiento

Para mantener el tiempo de entrenamiento razonable, se usan 20 épocas para entrenar el modelo (el modelo ha tenido la oportunidad de aprender de cada muestra en el conjunto de entrenamiento 20 veces)

En Colab, configura el entorno de ejecución a GPU para un entrenamiento más rápido.

In [43]:
# Constante 'EPOCHS' se configura en 20
EPOCHS = 20

In [44]:
# Se inicia el proceso de entrenamiento del modelo utilizando el conjunto de datos y se configura el número de épocas y los callbacks.
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])


Epoch 1/20
172/172 [==============================] - 15s 58ms/step - loss: 2.7573
Epoch 2/20
172/172 [==============================] - 11s 51ms/step - loss: 2.0146
Epoch 3/20
172/172 [==============================] - 12s 52ms/step - loss: 1.7315
Epoch 4/20
172/172 [==============================] - 11s 53ms/step - loss: 1.5675
Epoch 5/20
172/172 [==============================] - 12s 56ms/step - loss: 1.4659
Epoch 6/20
172/172 [==============================] - 12s 55ms/step - loss: 1.3965
Epoch 7/20
172/172 [==============================] - 13s 55ms/step - loss: 1.3428
Epoch 8/20
172/172 [==============================] - 13s 58ms/step - loss: 1.2976
Epoch 9/20
172/172 [==============================] - 12s 56ms/step - loss: 1.2565
Epoch 10/20
172/172 [==============================] - 12s 58ms/step - loss: 1.2181
Epoch 11/20
172/172 [==============================] - 13s 58ms/step - loss: 1.1790
Epoch 12/20
172/172 [==============================] - 13s 60ms/step - loss: 1.1395
E

En resumen, el anterior fragmento de código inicia el entrenamiento del modelo utilizando el conjunto de datos `dataset` y el número de épocas especificado en `EPOCHS`. También configura el entrenamiento para que guarde los checkpoints del modelo después de cada época utilizando el callback `checkpoint_callback`. La información sobre el proceso de entrenamiento se guarda en la variable `history`.

## Generación de texto

La manera más sencilla de generar texto con este modelo es correrlo en bucle, y mantener seguimiento al estado interno del modelo mientras se ejecuta.

![Para generar texto, la salida del modelo se introduce de nuevo como entrada](https://github.com/tensorflow/text/blob/master/docs/tutorials/images/text_generation_sampling.png?raw=1)

Cada vez que se llama el modelo se le pasa un texto y un estado interno. El modelo retorna una predicción del siguiente caracter y su nuevo estado. Se pasa la predicción y el estado de nuevo al modelo para seguir generando texto.

El siguiente código hace una predicción de una sola iteración:

In [45]:
# Se define una clase 'OneStep' que extiende 'tf.keras.Model' para la generación de texto.
class OneStep(tf.keras.Model):

  # Método de inicialización de la clase.
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()  # Llama al método de inicialización de la superclase.
    self.temperature = temperature  # Configura la temperatura para modificar la probabilidad de las predicciones, seteada en 1.0.
    self.model = model  # Modelo previamente entrenado.
    self.chars_from_ids = chars_from_ids  # Función para obtener caracteres a partir de IDs.
    self.ids_from_chars = ids_from_chars  # Función para obtener IDs a partir de caracteres.

    # Crea una máscara para evitar que se genere "[UNK]" (desconocido).
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        values=[-float('inf')]*len(skip_ids),  # Pone un -infinito en cada índice no deseado.
        indices=skip_ids,
        dense_shape=[len(ids_from_chars.get_vocabulary())])  # Hace coincidir la forma con el vocabulario.
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)  # Convierte el tensor disperso en denso.

  # Método para generar un paso (carácter) basado en una entrada dada.
  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convierte las cadenas en IDs de token.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Ejecuta el modelo. Obtiene los logits predichos y los estados.
    predicted_logits, states = self.model(inputs=input_ids, states=states, return_state=True)
    predicted_logits = predicted_logits[:, -1, :]  # Solo usa la última predicción.
    predicted_logits = predicted_logits/self.temperature  # Modifica los logits según la temperatura.

    # Aplica la máscara de predicción para evitar generar "[UNK]".
    predicted_logits = predicted_logits + self.prediction_mask

    # Muestrea los logits de salida para generar IDs de token.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)  # Elimina la dimensión innecesaria.

    # Convierte de IDs de token a caracteres.
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Devuelve los caracteres predichos y el estado del modelo.
    return predicted_chars, states


In [46]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

Se ejecuta en bucle para generar texto. Al observar el texto generado, se ve que el modelo sabe cuand poner una mayúscula, hacer párrafos e imitar vocabulario como el que usa Shakespeare. Con un pequeño número de épocas de entrenamiento, aún no aprende a cómo formar oraciones coherentes.

In [47]:
# Se registra el tiempo inicial para medir cuánto tiempo toma el proceso de generación.
start = time.time()

# Se inicializa el estado del modelo a 'None'.
states = None
# Se establece el carácter inicial como 'ROMEO:'.
next_char = tf.constant(['ROMEO:'])
# Se inicializa una lista con el carácter inicial para almacenar el resultado generado.
result = [next_char]

# Se generan 1000 caracteres usando el modelo 'one_step_model'.
for n in range(1000):
  # Se genera el siguiente carácter y se actualiza el estado del modelo.
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  # Se añade el carácter generado a la lista de resultados.
  result.append(next_char)

# Se unen todos los caracteres generados en una sola cadena.
result = tf.strings.join(result)

# Se registra el tiempo final.
end = time.time()

# Se imprime el texto generado y el tiempo que tomó el proceso.
#'result[0].numpy().decode('utf-8') convierte el tensor de TensorFlow a una cadena de Python.
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)


ROMEO:
Cord herrow from an old tale both with thee?

JULIET:

DUKE VINCENTIO:
Thou liest! come, dwell up your met,
Unfinister'd to a shrewdlike of
them. Camillo term these friends? Your Mowbray fight!
O, she is bold of sorrow and to Causes Warwick's daughter
Cloudy colours in his father's legs that very virtue.
I thought thou hadst breathed steel your signories:
Baiders war with Mowbray restend, I tender
And spirit to any of my troth, my
Away before you was ne'er to bower. Pray, girl:
Contempt at thy wearth! blest see an envy
Was, senators will request your grace.

GLOUCESTER:
My lord, your brother are my womanish fear:
Go, counted so the wars.

Ligut Watchman:
Here, sir! My blood should could be of forward;
On Titus, O fair issue!

LEONTES:
Once more!' the virty nor hold?

CLAUDIO:
Ay, sir; sound, sir, it is a wolves to be a male to love,
I come in bring their coward.

MERCUTIO:
A laughter; yet hath promised me worthy thief,
See, wast the tale as thy sacred spring-of blins,
An envious

La manera más sencilla para mejorar los resultados es entrenar el modelo por más epocas (intente `EPOCHS = 30`).

También puede experimentarse comenzando con una entrada diferente, añadiendo otra capa de RNN para mejorar la precisión del modelo, o ajustar la termperatura para generar predicciones más o menos aleatorias.

Si se quiere que el modelo produzca testo *más rápido*, la manera más sencilla es lotear ("batch") la generación de texto. En el ejemplo de abajo, el modelo genera 5 resultados en el mismo tiempo que toma generar 1 en las líneas anteriores.

In [48]:
start = time.time()
states = None

# Se establecen cinco cadenas iniciales, todas con el valor 'ROMEO:'.
next_char = tf.constant(['ROMEO:', 'ROMEO:', 'ROMEO:', 'ROMEO:', 'ROMEO:'])
result = [next_char]

# Se generan 1000 caracteres para cada una de las cadenas iniciales usando el modelo 'one_step_model'.
for n in range(1000):
  # Se generan los siguientes caracteres y se actualiza el estado del modelo.
  # Dado que hay cinco cadenas iniciales, en cada iteración se generan cinco caracteres.
  next_char, states = one_step_model.generate_one_step(next_char, states=states)

  # Se añaden los caracteres generados a la lista de resultados.
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result, '\n\n' + '_'*80)
print('\nRun time:', end - start)

tf.Tensor(
[b"ROMEO:\nGood ladies, you being there beastly, speak 'tis great so hate\nShall commend me to your jaultion,\nIn pardonious very prey. Now arm yourself.\n\nMISTRESS OVERDONE:\nWhats, marry, master, marry, as he had said,\nWith tongue is bounded to his sacred blood,\nWithout awaken upon our fair equally.\nThink up your subjects?\n\nSecond Murderer:\nI did; but all this waiting,\nWhere bloody will we marry her; and the\nserving-mere, is an envious eye.\n\nQUEEN ELIZABETH:\nWilt thou buside behind of rose,\nSo with my hate to save my mind, eat no\nnot Kate and weep not, not deceived.\n\nKING RICHARD II:\nA periorafe-bond I spake, and not woo'-take.\n\nVIRGILIA:\nIndeed, mine old fast for a lie:\nYou're for the entertain thus.\n\nDUKE VINCENTIO:\nThere is our general.\n\nMARCIUS:\nI go, my father, I have been well to revel me in hatrate\nAnd we must purchase till thee.\nI must be gone and not advance these power\nTo do your good time out of the sacrament,\nAnd privy to the peni

## Exportar el generador

Este modelo de un solo paso puede ser [ guardado y restaurado](https://www.tensorflow.org/guide/saved_model) fácilmente, lo que permite usarlo en cualquier lugar donde se acepte un `tf.saved_model`.



In [49]:
# Guardar el modelo 'one_step_model' en el directorio 'one_step' usando el formato 'SavedModel' de TensorFlow.
tf.saved_model.save(one_step_model, 'one_step')

# Cargar el modelo previamente guardado desde el directorio 'one_step' y almacenarlo en la variable 'one_step_reloaded'.
one_step_reloaded = tf.saved_model.load('one_step')

In [51]:
states = None
next_char = tf.constant(['JULIET:'])
result = [next_char]

for n in range(500):
  next_char, states = one_step_reloaded.generate_one_step(next_char, states=states)
  result.append(next_char)

print(tf.strings.join(result)[0].numpy().decode("utf-8"))

JULIET:
I hope here were the case?
Your methority, our brother's love, penuse,
And ture as strong aside; and so,
I hear she bloody in a band, o'
Who is lost by, to say I cannot, bear
A moider where 'tis awaked with main-maid!
Besides, I heard the Duke of Norfolk sinser Marcius;
One, one part of wide, and give sick
And to be but verited with our field
In yet in peace and most abrect
Ane seas thrust must action: thy general.

FERDINAND:
No;
I am now--wilt thou mine own likeness. Come hither, Stanley and 


## Avanzado: Personalizar entrenamiento

El procedimiento de entrenamiento anterior es simple, pero no brinda mucho control. Se utiliza el método de teacher-forcing, que evita que las predicciones erróneas se retroalimenten al modelo, por lo que el modelo nunca aprende de los errores.

Ahora que se ha visto cómo ejecutar el modelo manualmente, a continuación se implementara el bucle de entrenamiento. Esto da un punto de partida si, por ejemplo, se desea implementar _aprendizaje por niveles_ para ayudar a estabilizar la salida en bucle abierto del modelo.

La parte más importante de un bucle de entrenamiento personalizado es la función "paso de entrenamiento", que se refiere a una iteración donde el modelo procesa un lote de datos y actualiza sus pesos.

Se usa `tf.GradientTape` para rastrear los gradientes. Puede aprender más sobre este enfoque leyendo la guía [eager execution guide](https://www.tensorflow.org/guide/eager).

El procedimiento básico es:

1. Ejecutar el modelo y calcular la pérdida bajo un `tf.GradientTape`.
2. Calcular las actualizaciones y aplicarlas al modelo utilizando el optimizador.

In [52]:
# Se define una clase 'CustomTraining' que hereda de 'MyModel'.
class CustomTraining(MyModel):

  # Se utiliza el decorador '@tf.function' para convertir la función siguiente en una función TensorFlow (para optimizar su ejecución).
  @tf.function
  def train_step(self, inputs):

      # Se desempaquetan los datos de entrada en variables 'inputs' y 'labels'.
      # 'inputs' son los datos que se alimentan al modelo, y 'labels' son las etiquetas verdaderas que se usarán para calcular la pérdida.
      inputs, labels = inputs

      # Se inicia un contexto 'tf.GradientTape' para registrar operaciones para la diferenciación automática.
      with tf.GradientTape() as tape:

          # Se obtienen las predicciones del modelo para los datos de entrada.
          # El parámetro 'training=True' indica que el modelo está en modo de entrenamiento.
          predictions = self(inputs, training=True)

          # Se calcula la pérdida entre las etiquetas verdaderas y las predicciones del modelo.
          loss = self.loss(labels, predictions)

      # Se calculan los gradientes de la pérdida con respecto a las variables entrenables del modelo.
      grads = tape.gradient(loss, model.trainable_variables)

      # Se aplican los gradientes a las variables entrenables utilizando el optimizador del modelo.
      self.optimizer.apply_gradients(zip(grads, model.trainable_variables))

      # Se retorna un diccionario con el valor de la pérdida.
      return {'loss': loss}


La anterior implementación del método `train_step` sigue las [Convenciones de entrenamiento `train_step` de Keras](https://www.tensorflow.org/guide/keras/customizing_what_happens_in_fit). Esto es opcional, pero permite cambiar el comportamiento del paso de entrenamiento y continuar usando los métodos de keras `Model.compile` y `Model.fit`.

In [53]:
model = CustomTraining(
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

In [54]:
model.compile(optimizer = tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True))

In [55]:
model.fit(dataset, epochs=1)

172/172 [==============================] - 14s 55ms/step - loss: 2.7482


O, de necesitar más control, puede escribir su propio bucle de entrenamiento personalizado:

In [ ]:
# Se establece el número total de épocas para el entrenamiento.
EPOCHS = 10

# Se crea una métrica para calcular la media de la pérdida.
mean = tf.metrics.Mean()

# Se inicia el bucle de entrenamiento principal para todas las épocas.
for epoch in range(EPOCHS):

    # Se registra el tiempo de inicio de la época para calcular cuánto tiempo toma.
    start = time.time()

    # Se reinician los estados de la métrica 'mean' al inicio de cada época.
    mean.reset_states()

    # Se itera sobre el conjunto de datos 'dataset', que contiene pares de entrada y objetivo.
    for (batch_n, (inp, target)) in enumerate(dataset):

        # Se realiza un paso de entrenamiento usando el método 'train_step' del modelo y se obtienen los registros (logs).
        logs = model.train_step([inp, target])

        # Se actualiza la métrica 'mean' con la pérdida del paso actual.
        mean.update_state(logs['loss'])

        # Cada 50 lotes, se imprime la pérdida actual.
        if batch_n % 50 == 0:
            template = f"Epoch {epoch+1} Batch {batch_n} Loss {logs['loss']:.4f}"
            print(template)

    # Cada 5 épocas, se guarda el modelo.
    if (epoch + 1) % 5 == 0:
        model.save_weights(checkpoint_prefix.format(epoch=epoch))

    # Al final de cada época, se imprime un resumen con la pérdida media y el tiempo que tomó.
    print()
    print(f'Epoch {epoch+1} Loss: {mean.result().numpy():.4f}')
    print(f'Time taken for 1 epoch {time.time() - start:.2f} sec')
    print("_"*80)

# Después de completar todas las épocas, se guarda el modelo una vez más.
model.save_weights(checkpoint_prefix.format(epoch=epoch))